In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
# The token can be used to access the repository
!git clone https://github.com/cariello1/BIO-NER-.git

Cloning into 'BIO-NER-'...
remote: Enumerating objects: 493, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 493 (delta 75), reused 0 (delta 0), pack-reused 365
Receiving objects: 100% (493/493), 12.49 MiB | 24.40 MiB/s, done.
Resolving deltas: 100% (261/261), done.


In [ ]:
%cd /content/BIO-NER-

/content/BIO-NER-


In [ ]:
from preprocessing_1 import PreprocessingDocsCRF
from preprocessing_1 import read_data

# PREPROCESSING

In [ ]:
import numpy as np
import seaborn as sns
sns.set_style("whitegrid")
from matplotlib import pyplot as plt
import tqdm

In [ ]:
# Original data in IOB format
file1 = "/content/gdrive/MyDrive/Data_Bio-NER/JNLPBA/original-data/train/Genia4ERtask1.iob2"
file2 = "/content/gdrive/MyDrive/Data_Bio-NER/JNLPBA/original-data/test/Genia4EReval1.iob2"

In [ ]:
input_train = read_data(file1)
input_test = read_data(file2)

Total number of sentences: 18546 
Total number of sentences: 3856 


In [ ]:
docs_train = PreprocessingDocsCRF(input_train)
data_train_withPOS = docs_train.addPOStags()

1it [00:00,  8.30it/s]

Adding part-of-speech tags to the original corpus (word, label)...


18546it [00:23, 794.69it/s]

Done.


In [ ]:
docs_train

In [ ]:
docs_test = PreprocessingDocsCRF(input_test)
data_test_withPOS = docs_test.addPOStags()

77it [00:00, 763.65it/s]

Adding part-of-speech tags to the original corpus (word, label)...


3856it [00:04, 801.60it/s]

Done.


In [ ]:
# First sentence of train dataset with word, label, POStag
print('This is what a sentence looks like:{}'.format(data_train_withPOS[0]))

This is what a sentence looks like:[('IL-2', 'ADJ', 'B-DNA'), ('gene', 'NOUN', 'I-DNA'), ('expression', 'NOUN', 'O'), ('and', 'CONJ', 'O'), ('NF-kappa', 'NOUN', 'B-protein'), ('B', 'NOUN', 'I-protein'), ('activation', 'NOUN', 'O'), ('through', 'ADP', 'O'), ('CD28', 'NOUN', 'B-protein'), ('requires', 'VERB', 'O'), ('reactive', 'ADJ', 'O'), ('oxygen', 'NOUN', 'O'), ('production', 'NOUN', 'O'), ('by', 'ADP', 'O'), ('5-lipoxygenase', 'ADJ', 'B-protein'), ('.', '.', 'O')]


In [ ]:
import string 
punct = string.punctuation
def hasPunct(string):
  return any(char for char in string if char in punct)

In [ ]:
def hasNumbers(string):
  return any(char.isdigit() for char in string)

In [ ]:
from collections import Counter
word_freq = Counter(docs_train.tokens)

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
word_freq_purify = {key:value for (key,value) in word_freq.items() if key.lower() not in stopwords.words('english') }

In [ ]:
def word_freq(word):
  try: 
    value = word_freq_purify["word"]
  except:
    value = 0
  return value


# FEATURES SET

Morphological features: 

*    Is-Capitalized, Is-Title, Contains-Digits, Contains-Punctuations (Boolean)
*   Prefixes, Suffixes (Nominal)
*   Word's length, Term Frequency (Numeric)

Lexical features: 

*   POS Tagging (Nominal)

Some information from **nearby words** is added, as well as all these features for words that are not at the beginning of a document (BOS), all these features for words that are not at the end of a document (EOS).

Contextual information (-2/+2 token window):

*   Is-Title, Is-Capitalized, Contains-Digits, Contains-Punctuations, Word's length, POS Tagging



In [ ]:
 def word2features(sent, i):
    """Map words to dict of features. Code from sklearn-crfsuite API documentation."""
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'wordID': word, #WordIdentity
        'word[-3:]': word[-3:], #suffix
        'word[-2:]': word[-2:],
        'word[:3]': word[:3], #prefix
        'word[:2]': word[:2],
        'word.isupper()': word.isupper(), #isCapitalized
        'hasPunct(word)': hasPunct(word),
        'hasNumber(word)': hasNumbers(word), #ContainsDigits
        'word.istitle()': word.istitle(),
        'len(word)': len(word), #word's length
        'word_freq(word)': word_freq(word), #term frequency
        'postag': postag
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:wordID': word1,
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:hasPunct(word)': hasPunct(word),
            '-1:hasNumber(word)': hasNumbers(word),
            '-1:postag': postag1,
            '-1:len(word)': len(word)
        })

    if i > 1:
        word1 = sent[i-2][0]
        postag1 = sent[i-2][1]
        features.update({
            '-2:wordID': word1,
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper(),
            '-2:hasPunct(word)': hasPunct(word),
            '-2:hasNumber(word)': hasNumbers(word),
            '-2:postag': postag1,
            '-2:len(word)': len(word)    
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:wordID': word1,
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:hasPunct(word)': hasPunct(word),
            '+1:hasNumber(word)': hasNumbers(word),
            '+1:postag': postag1,
            '+1:len(word)': len(word) 
        })

    if i < len(sent)-2:
        word1 = sent[i+2][0]
        postag1 = sent[i+2][1]
        features.update({
            '+2:wordID': word1,
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper(),
            '+2:hasPunct(word)': hasPunct(word),
            '+2:hasNumber(word)': hasNumbers(word),
            '+2:postag': postag1,
            '+2:len(word)': len(word) 
        })
    else:
        features['EOS'] = True

    return features

In [ ]:
def sent2features(sent):
    """Map sent to dict of features for each word. Code from sklearn-crfsuite API documentation."""
    return [word2features(sent, i) for i in range(len(sent))]

In [ ]:
#from preprocessing_1 import sent2features
# Transform a list of sentences in a list of features for each word in a sentence
X_train = [sent2features(s) for s in data_train_withPOS]
X_test = [sent2features(s) for s in data_test_withPOS]

In [ ]:
# Feature dict for the second word of the first sent in test set
X_test[0][2]

{'+1:hasNumber(word)': False,
 '+1:hasPunct(word)': False,
 '+1:len(word)': 14,
 '+1:postag': 'NOUN',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:wordID': 'receptors',
 '+2:hasNumber(word)': False,
 '+2:hasPunct(word)': False,
 '+2:len(word)': 14,
 '+2:postag': 'ADP',
 '+2:word.istitle()': False,
 '+2:word.isupper()': False,
 '+2:wordID': 'in',
 '-1:hasNumber(word)': False,
 '-1:hasPunct(word)': False,
 '-1:len(word)': 14,
 '-1:postag': 'ADP',
 '-1:word.istitle()': False,
 '-1:word.isupper()': False,
 '-1:wordID': 'of',
 '-2:hasNumber(word)': False,
 '-2:hasPunct(word)': False,
 '-2:len(word)': 14,
 '-2:postag': 'NOUN',
 '-2:word.istitle()': True,
 '-2:word.isupper()': False,
 '-2:wordID': 'Number',
 'bias': 1.0,
 'hasNumber(word)': False,
 'hasPunct(word)': False,
 'len(word)': 14,
 'postag': 'ADJ',
 'word.istitle()': False,
 'word.isupper()': False,
 'wordID': 'glucocorticoid',
 'word[-2:]': 'id',
 'word[-3:]': 'oid',
 'word[:2]': 'gl',
 'word[:3]': 'glu',
 'word_f

In [ ]:
# Transform a list of lbel sentences in a list of labels for each word in a sentence
y_train = [[l for w, pos, l in s] for s in data_train_withPOS]
y_test = [[l for w, pos, l in s] for s in data_test_withPOS]

In [ ]:
y_test[0]

['O',
 'O',
 'B-protein',
 'I-protein',
 'O',
 'B-cell_type',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

# TRAINING

In [ ]:
%%capture
# python-crfsuite wrapper compatible with sklearn
!pip install sklearn_crfsuite 

In [ ]:
from sklearn_crfsuite import CRF

In [ ]:
crf_model = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

In [ ]:
%%time
crf_model.fit(X_train, y_train)

CPU times: user 2min 18s, sys: 637 ms, total: 2min 18s
Wall time: 2min 18s


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

# PREDICTION

In [ ]:
%%time
y_hat = crf_model.predict(X_test)

CPU times: user 2.22 s, sys: 23.8 ms, total: 2.24 s
Wall time: 2.26 s


In [ ]:
y_hat

[['O',
  'O',
  'B-protein',
  'I-protein',
  'O',
  'B-cell_type',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-protein',
  'I-protein',
  'O',
  'B-protein',
  'O',
  'O',
  'B-cell_type',
  'I-cell_type',
  'I-cell_type',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'B-cell_type',
  'O',
  'O',
  'O',
  'B-protein',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-cell_type',
  'I-cell_type',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-protein',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-cell_type',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-protein'

# REPORT

In [ ]:
%%capture
# Install seqeval library for evaluation score
!pip3 install seqeval

In [ ]:
from seqeval.metrics import f1_score, classification_report

In [ ]:
from seqeval.scheme import IOB2
print('\033[1m' + "Classification report for BERT: ")
print('\033[0m')
print(classification_report(y_test, y_hat,scheme=IOB2, mode="strict"))

Classification report for BERT: 

              precision    recall  f1-score   support

         DNA       0.66      0.63      0.64      1056
         RNA       0.66      0.69      0.67       118
   cell_line       0.53      0.58      0.56       500
   cell_type       0.78      0.64      0.70      1921
     protein       0.67      0.74      0.70      5067

   micro avg       0.68      0.69      0.69      8662
   macro avg       0.66      0.65      0.66      8662
weighted avg       0.69      0.69      0.69      8662



In [ ]:
print("F1-score: {:0.3}".format(f1_score(y_test, y_hat, average = "micro", scheme=IOB2, mode="strict")))

F1-score: 0.686


In [ ]:
%%capture
!pip install nervaluate

In [ ]:
from nervaluate import Evaluator
golden_tags = ["protein", "cell_type", "cell_line", "DNA", "RNA"]
evaluator = Evaluator(y_test, y_hat, tags = golden_tags, loader="list")
results, results_by_tag = evaluator.evaluate()
import pandas as pd
res = pd.DataFrame.from_dict(results)

In [ ]:
res

,ent_type,partial,strict,exact
correct,6853.000000,6347.000000,5992.000000,6347.000000
incorrect,553.000000,0.000000,1414.000000,1059.000000
partial,0.000000,1059.000000,0.000000,0.000000
missed,1256.000000,1256.000000,1256.000000,1256.000000
spurious,1402.000000,1402.000000,1402.000000,1402.000000
possible,8662.000000,8662.000000,8662.000000,8662.000000
actual,8808.000000,8808.000000,8808.000000,8808.000000
precision,0.778043,0.780711,0.680291,0.720595
recall,0.791157,0.793870,0.691757,0.732741
f1,0.784545,0.787235,0.685976,0.726617


In [ ]:
results_by_tag["DNA"]["strict"]

{'actual': 1093,
 'correct': 662,
 'f1': 0.6161005118659842,
 'incorrect': 254,
 'missed': 140,
 'partial': 0,
 'possible': 1056,
 'precision': 0.605672461116194,
 'recall': 0.6268939393939394,
 'spurious': 177}

In [ ]:
results_by_tag["DNA"]["partial"]

{'actual': 1093,
 'correct': 772,
 'f1': 0.785481619357841,
 'incorrect': 0,
 'missed': 140,
 'partial': 144,
 'possible': 1056,
 'precision': 0.7721866422689845,
 'recall': 0.7992424242424242,
 'spurious': 177}

In [ ]:
results_by_tag["protein"]["strict"]

{'actual': 5410,
 'correct': 3735,
 'f1': 0.712990359835831,
 'incorrect': 646,
 'missed': 686,
 'partial': 0,
 'possible': 5067,
 'precision': 0.6903881700554528,
 'recall': 0.7371225577264654,
 'spurious': 1029}

In [ ]:
results_by_tag["protein"]["partial"]

{'actual': 5410,
 'correct': 3785,
 'f1': 0.7794215901498521,
 'incorrect': 0,
 'missed': 686,
 'partial': 596,
 'possible': 5067,
 'precision': 0.7547134935304991,
 'recall': 0.8058022498519835,
 'spurious': 1029}

In [ ]:
results_by_tag["RNA"]["strict"]

{'actual': 134,
 'correct': 81,
 'f1': 0.6428571428571428,
 'incorrect': 30,
 'missed': 7,
 'partial': 0,
 'possible': 118,
 'precision': 0.6044776119402985,
 'recall': 0.6864406779661016,
 'spurious': 23}

In [ ]:
results_by_tag["RNA"]["partial"]

{'actual': 134,
 'correct': 93,
 'f1': 0.8095238095238095,
 'incorrect': 0,
 'missed': 7,
 'partial': 18,
 'possible': 118,
 'precision': 0.7611940298507462,
 'recall': 0.864406779661017,
 'spurious': 23}

In [ ]:
results_by_tag["cell_line"]["strict"]

{'actual': 468,
 'correct': 289,
 'f1': 0.597107438016529,
 'incorrect': 124,
 'missed': 87,
 'partial': 0,
 'possible': 500,
 'precision': 0.6175213675213675,
 'recall': 0.578,
 'spurious': 55}

In [ ]:
results_by_tag["cell_line"]["partial"]

{'actual': 468,
 'correct': 340,
 'f1': 0.7778925619834711,
 'incorrect': 0,
 'missed': 87,
 'partial': 73,
 'possible': 500,
 'precision': 0.8044871794871795,
 'recall': 0.753,
 'spurious': 55}

In [ ]:
results_by_tag["cell_type"]["strict"]

{'actual': 1703,
 'correct': 1225,
 'f1': 0.6760485651214128,
 'incorrect': 360,
 'missed': 336,
 'partial': 0,
 'possible': 1921,
 'precision': 0.7193188490898414,
 'recall': 0.6376887038001041,
 'spurious': 118}

In [ ]:
results_by_tag["cell_type"]["partial"]

{'actual': 1703,
 'correct': 1357,
 'f1': 0.8118101545253863,
 'incorrect': 0,
 'missed': 336,
 'partial': 228,
 'possible': 1921,
 'precision': 0.8637698179682912,
 'recall': 0.7657470067673087,
 'spurious': 118}

#  OPTIMIZATION

Implemented following sklearn-crfsuite documentation by Mikhail Korobov.

In [ ]:
#import scipy
#parameter_space = {'c1': scipy.stats.expon(scale=0.5),
  #  'c2': scipy.stats.expon(scale=0.5)}

In [ ]:
from sklearn.metrics import make_scorer
from sklearn_crfsuite import metrics
from sklearn.model_selection import GridSearchCV

# Wrapped f1_score in a customized function
def my_scorer1(y, y_pred):
 # y_hat, y_true = ignoring_padding(y_pred, y)
  return f1_score(y, y_pred, average = "micro", scheme=IOB2, mode="strict")

# Use the same metric for evaluation
#f1_scorer = make_scorer(my_scorer1)
"""
# Search
grid = GridSearchCV(CRF(verbose=10), parameters,
                        cv=3,
                        verbose=10,
                        n_jobs=-1,
                        scoring=f1_scorer)"""

'\n# Search\ngrid = GridSearchCV(CRF(verbose=10), parameters,\n                        cv=3,\n                        verbose=10,\n                        n_jobs=-1,\n                        scoring=f1_scorer)'

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(CRF(), parameter_space,
                        cv=3,
                        verbose=10,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)

NameError: ignored

In [ ]:
%%time
#rs.fit(X_train, y_train)

In [ ]:
rs.best_params_

In [ ]:
%%time
#grid.fit(X_train, y_train, verbose = 10)

In [ ]:
while True: pass

In [ ]:
#crf = rs.best_estimator_

In [ ]:
# crf = rs.best_estimator_
"""print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))"""

In [ ]:
# Save the model to disk
import joblib
filename = "/content/gdrive/MyDrive/BIO-NER-project/crf_model.sav"
joblib.dump(crf_model, filename)

In [ ]:
# Load the model
loaded_model = joblib.load(filename)

In [ ]:
y_hat = loaded_model.predict(X_test)

In [ ]:
print('\033[1m' + "Classification report for simple linear CRF model (optimized): ")
print('\033[0m')
print(classification_report(y_test, y_hat))

In [ ]:
print("F1-score(relaxed): {:0.3}".format(f1_score(y_test, y_hat, average = "weighted")))

In [ ]:
print('\033[1m' + "Classification report for simple linear CRF model (optimized): "
print('\033[0m')
print(classification_report(y_test, y_hat, scheme=IOB2, mode="strict"))

In [ ]:
print("F1-score: {:0.3}".format(f1_score(y_test, y_hat, average = "weighted", scheme=IOB2, mode="strict")))

# DEBUGGING WITH ELI5

In [ ]:
%%capture
!pip3 install eli5

In [ ]:
import eli5
eli5.show_weights(crf_model, top=10)

In [ ]:
from collections import Counter

def print_transitions(trans_features):
  for (label_from, label_to), weight in trans_features:
    print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))


In [ ]:
print("Top likely transitions:")
print_transitions(Counter(crf_model.transition_features_).most_common(20))

In [ ]:
print("\nTop unlikely transitions:")
print_transitions(Counter(crf_model.transition_features_).most_common()[-20:])

# COMPARE RESULTS

In [ ]:
# Compare results
i = 18
print("{:15} {:5}: ({})".format("Word", "Pred", "True"))
print("="*45)
for w, true, pred in zip(docs_test.sentences[i], y_test[i], y_hat[i]):
      print("{:15}:{:15} ({})".format(w, pred, true))

Word            Pred : (True)
However        :O               (O)
,              :O               (O)
there          :O               (O)
was            :O               (O)
a              :O               (O)
significantly  :O               (O)
positive       :O               (O)
association    :O               (O)
between        :O               (O)
tumor          :B-protein       (B-protein)
beta           :I-protein       (I-protein)
2-M            :I-protein       (I-protein)
expression     :O               (O)
and            :O               (O)
the            :O               (O)
degree         :O               (O)
of             :O               (O)
lymphocytic    :O               (O)
infiltration   :O               (O)
in             :O               (O)
the            :O               (O)
tumor          :O               (O)
tissue         :O               (O)
.              :O               (O)
